# Stock Predict Model

## Set Up

### Import Modules

In [693]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# For Graph
import matplotlib.pyplot as plt

# Pytorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

# Yahoo API
import yfinance as yf

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Configurations

In [769]:
device = 'cpu' if torch.backends.mps.is_available() else 'cpu'
scaler = StandardScaler()
config = {
    'seed': 666999,
    'select_all': False,
    'valid_ratio': 0.2,
    'test_ratio': 0.2,
    'n_epochs': 3000,
    'learning_rate': 1e-5,
    'early_stop': 300,
    'save_path': './models/stock.ckpt',
    'data_loader': {
        'batch_size': 130,
        'pin_memory': True,
        'num_workers': 0
    }
}

### Utility Functions

In [821]:
def same_seed(seed):
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

def predict(test_loader, model, device):
    model.eval()
    result = {'features': np.array([[]]), 'preds': np.array([]), 'ans': np.array([])}
    for x, y in tqdm(test_loader):
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            pred = model(x)
            print(x.detach().cpu().numpy().shape)
            result['features'] = np.append(result['features'], np.array(x.detach().cpu().numpy()))
            result['preds'] = np.append(result['preds'], np.array(pred.detach().cpu().numpy()))
            result['ans'] = np.append(result['ans'], np.array(y.detach().cpu().numpy()))

    print(result['features'].shape)
    print(result['preds'].shape)
    print(result['ans'].shape)
    predstack = np.column_stack((result['features'].reshape(1,-1), result['preds']))
    targetstack = np.column_stack((result['features'].reshape(1,-1), result['ans']))

    # Inver Normalized
    predstack = scaler.inverse_transform(predstack)
    targetstack = scaler.inverse_transform(targetstack)

    # preds = torch.cat(torch.from_numpy(result['preds']), dim=0).numpy()
    return np.column_stack((predstack[:, -1], targetstack[:, -1]))

## Data

### Dataset

In [771]:
class Stock_Dataset(Dataset):
    '''
    x: Features.
    y: Targets, if none, do prediction.
    '''
    def __init__(self, x, y=None, normalized=True, initscaler=True):
        # Normalized
        stack = np.column_stack((x, y))

        if (normalized):
            normalstack = scaler.fit_transform(stack) if initscaler else scaler.transform(stack)
            x = normalstack[:, :-1]
            y = y if y is None else normalstack[:, -1]

        # x = scaler.fit_transform(x) if normalized else x
        # if (y is not None):
        #     y = scaler.fit_transform(y.reshape(-1, 1)) if normalized else y

        self.y = y if y is None else torch.FloatTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

### DataLoader

In [772]:
def stock_dataloader(config, x, y=None, normalized=True, initscaler=True):
    ''' Generates a dataset, then is put into a dataloader. '''

    x = np.array(x)
    y = np.array(y) if y is not None else y

    dataset = Stock_Dataset(x, y, normalized)
    dataloader = DataLoader(dataset, **config['data_loader'])

    return dataloader

### Feature Selection

In [763]:
def select_feat(train_data, valid_data, test_data, select_all=True):
    '''Selects useful features to perform regression'''
    y_train, y_valid, y_test = train_data[:,-1], valid_data[:,-1], test_data[:, -1]
    raw_x_train, raw_x_valid, raw_x_test = train_data[:, :-1], valid_data[:, :-1], test_data[:, :-1]

    if select_all:
        feat_idx = list(range(raw_x_train.shape[1]))
    else:
        feat_idx = [0,1,2,3] # TODO: Select suitable feature columns.

    return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid, y_test

### Set Data

In [764]:
same_seed(config['seed'])

# re-index
index = ["Open", "High", "Low", "Adj Close", "Volume", "Close"]
raw_data = pd.DataFrame(yf.download('2330.TW', start='2000-01-01', end='2023-12-31'))
# raw_data.to_csv("raw.csv", index=False)
train_data, test_data = train_test_split(
    raw_data[index].values,
    test_size=config['test_ratio'],
    random_state=config['seed'],
    shuffle=False
)

train_data, valid_data = train_test_split(
    train_data,
    test_size=config['valid_ratio'],
    random_state=config['seed'],
    shuffle=True
)

# Print out the data size.
print(f"""train_data size: {train_data.shape}
valid_data size: {valid_data.shape}
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid, y_test = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

# Pytorch data loader loads pytorch dataset into batches.
train_loader = stock_dataloader(config, x_train, y_train)
valid_loader = stock_dataloader(config, x_valid, y_valid, initscaler=False)
test_loader = stock_dataloader(config, x_test, y_test, initscaler=False)

[*********************100%***********************]  1 of 1 completed
train_data size: (3735, 6)
valid_data size: (934, 6)
test_data size: (1168, 6)
number of features: 4


## Neural Network

### Model

In [700]:
class Stock_Model(nn.Module):
    def __init__(self, input_dim):
        super(Stock_Model, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        x = self.layers(x)
        x = x.squeeze(1)
        return x

### Training Loop

In [701]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.MSELoss(reduction='mean')
    optimizer = torch.optim.SGD(model.parameters(), lr=config['learning_rate'], momentum=0.7)

    writer = SummaryWriter()

    if not os.path.isdir('./models'):
        os.mkdir('./models')

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train()
        loss_record = []

        train_pbar = tqdm(train_loader, position=0, leave=True)
        for x, y in train_pbar:
            optimizer.zero_grad()
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            step += 1
            loss_record.append(loss.detach().item())

            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})
        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval()
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')
        writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Training

In [702]:
model = Stock_Model(input_dim=x_train.shape[1]).to(device)
trainer(train_loader, valid_loader, model, config, device)

# with torch.profiler.profile(
#     activities=[torch.profiler.ProfilerActivity.CPU],
#     schedule = torch.profiler.schedule(
#         wait=0,
#         warmup=0,
#         active=1
#     ),
#     on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
# ) as prof:
#     trainer(train_loader, valid_loader, model, config, device)
#     prof.step()

  0%|          | 0/29 [00:00<?, ?it/s]/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([130, 1])) that is different to the input size (torch.Size([130])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch [1/3000]:  93%|█████████▎| 27/29 [00:00<00:00, 36.74it/s, loss=0.788]/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([95, 1])) that is different to the input size (torch.Size([95])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
Epoch [1/3000]: 100%|██████████| 29/29 [00:00<00:00, 30.70it/s, loss=1.25] 
/usr/local/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([24,

Epoch [1/3000]: Train loss: 1.0562, Valid loss: 1.0199
Saving model with loss 1.020...


Epoch [2/3000]: 100%|██████████| 29/29 [00:00<00:00, 46.44it/s, loss=1.25] 


Epoch [2/3000]: Train loss: 1.0558, Valid loss: 1.0196
Saving model with loss 1.020...


Epoch [3/3000]: 100%|██████████| 29/29 [00:00<00:00, 31.70it/s, loss=1.25] 


...

Epoch [979/3000]: Train loss: 1.0017, Valid loss: 0.9793


Epoch [980/3000]: 100%|██████████| 29/29 [00:00<00:00, 49.61it/s, loss=1.13] 


Epoch [980/3000]: Train loss: 1.0017, Valid loss: 0.9793


Epoch [981/3000]: 100%|██████████| 29/29 [00:00<00:00, 49.16it/s, loss=1.13] 


Epoch [981/3000]: Train loss: 1.0017, Valid loss: 0.9793

Model is not improving, so we halt the training session.


In [703]:
# print(prof.key_averages().table(sort_by="self_cpu_time_total", row_limit=10))


In [704]:
%tensorboard --logdir=./log/

ERROR: Failed to launch TensorBoard (exited with -9).

## Plot learning curves with tensorboard

In [705]:
%reload_ext tensorboard
%tensorboard --logdir=./runs/

ERROR: Failed to launch TensorBoard (exited with -9).

## Testing

In [822]:
def save_pred(preds, file):
    ''' Save predictions to specified file '''
    np.savetxt(file, preds, delimiter=',', fmt='%d', header='pred,target', comments='')
    # with open(file, 'w') as fp:
    #     writer = csv.writer(fp)
    #     writer.writerow(['id', 'tested_positive'])
    #     for i, p in enumerate(preds):
    #         writer.writerow([i, p[0], p[1]])

model = Stock_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)
save_pred(preds, 'pred.csv')


100%|██████████| 9/9 [00:00<00:00, 612.29it/s]

(130, 4)
(130, 4)
(130, 4)
(130, 4)
(130, 4)
(130, 4)
(130, 4)
(130, 4)
(128, 4)
(4672,)
(1168,)
(1168,)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 1 and the array at index 1 has size 1168

In [816]:
model = Stock_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))

single_loader = stock_dataloader(config, [[100,110,90,101]], [103])
predict(single_loader, model, device)


100%|██████████| 1/1 [00:00<00:00, 536.49it/s]

torch.Size([1, 4])
(4,)
(1,)
(1,)


array([[102.95208965, 103.        ]])